In [2]:
import pandas as pd
import re
import math
import numpy as np
import scipy
import os
import time
import seaborn as sns
from scipy.stats import binom, hypergeom
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [15]:
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv('cab_rides.csv')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['day']=df.time_stamp.apply(lambda x: time.strftime('%a', time.localtime(x/1000)))


In [ ]:
df['AMPM']=df.time_stamp.apply(lambda x: time.strftime('%p', time.localtime(x/1000)))


In [ ]:
df['month']=df.time_stamp.apply(lambda x: time.strftime('%b', time.localtime(x/1000)))


In [ ]:
df['time']=df.time_stamp.apply(lambda x: time.strftime('%H', time.localtime(x/1000)))


In [ ]:
df['week']=df.time_stamp.apply(lambda x: time.strftime('%U', time.localtime(x/1000)))


In [ ]:
df['date_time']=df.time_stamp.apply(lambda x: time.strftime('%Y-%m-%d %H', time.localtime(x/1000)))


In [ ]:
df.date_time.max()

In [ ]:
df.date_time.min()

In [ ]:
df.week.max()

In [ ]:
df.week.min()

In [ ]:
df['merge_date'] = df.source.astype(str) +" - "+ df.date_time.astype("str") 



In [ ]:
df.head()

In [ ]:
df1 = pd.read_csv('weather.csv')

In [ ]:
df1.head()

In [ ]:
df1['date_time']=df1.time_stamp.apply(lambda x: time.strftime('%Y-%m-%d %H', time.localtime(x)))


In [ ]:
df1.date_time

In [ ]:
#merge the datasets to refelect same time for a location
df1['merge_date'] = df1.location.astype(str) +" - "+ df1.date_time.astype("str") 


In [ ]:
merged_df = df.merge(df1, how='inner', left_on = ['merge_date'], right_on=['merge_date'], suffixes=('_c','_w'))
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
merged_df.drop_duplicates(subset='id', keep='first', inplace=True)

In [ ]:
merged_df.info()

In [ ]:
merged_df['time'] = merged_df['time'].astype(int)


In [ ]:
merged_df['rushhr'] = 0

In [ ]:
merged_df.loc[(merged_df.time > 7) & (merged_df.time < 10), 'rushhr'] = 1

In [ ]:
merged_df.loc[(merged_df.time > 17) & (merged_df.time < 22), 'rushhr'] = 1

In [ ]:
merged_df.groupby('time')['rushhr'].sum()

In [ ]:
merged_df = pd.get_dummies(merged_df, columns=[
                           'cab_type', 'destination', 'source', 'name', 'day', 'AMPM', 'month', 'week'])

In [ ]:
merged_df.head(100)

In [ ]:
merged_df['Black'] = 0

In [ ]:
merged_df['Black'] = merged_df['name_Black'] + merged_df['name_Black SUV'] + \
    merged_df['name_Lux Black'] + merged_df['name_Lux Black XL']

In [ ]:
merged_df.groupby('Black')['name_Black','name_Black SUV','name_Lux Black','name_Lux Black XL'].sum()

In [ ]:
merged_df['sports'] = 0

In [ ]:
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-02')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-11-30')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-06')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-10')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-14')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-11-29')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-01')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-08')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-11')) & (merged_df.AMPM_PM == 1), 'sports'] = 1
merged_df.loc[(merged_df.date_time_c.str.contains('2018-12-16')) & (merged_df.AMPM_PM == 1), 'sports'] = 1

In [ ]:
sportdf = merged_df.groupby('date_time_c')['sports'].sum()
with pd.option_context("display.max_rows", 1000):
    display(sportdf)

In [ ]:
np.sum(merged_df.isnull(),axis = 0)

In [ ]:
merged_df.rain.fillna(0, inplace= True)

In [ ]:
merged_df['rain_d']=0

In [ ]:
merged_df.loc[(merged_df.rain) > 0, 'rain_d'] = 1


In [ ]:
merged_df[merged_df.rain_d == 0].count()

In [ ]:
merged_df['wkndPM']=0

In [ ]:
merged_df.loc[(merged_df.day_Fri == 1) & (merged_df.AMPM_PM == 1), 'wkndPM'] = 1
merged_df.loc[(merged_df.day_Sat == 1) & (merged_df.AMPM_PM == 1), 'wkndPM'] = 1
merged_df.loc[(merged_df.day_Sun == 1) & (merged_df.AMPM_PM == 1), 'wkndPM'] = 1

In [ ]:
wkndPMdf = merged_df.groupby('merge_date')['wkndPM'].sum()
with pd.option_context("display.max_rows", 1000):
    display(wkndPMdf)

In [ ]:
merged_df.dropna(inplace=True)

In [ ]:
merged_df.drop(['time_stamp_c', 'time_stamp_w', 'date_time_c',
                'date_time_w', 'id', 'product_id'], axis=1, inplace=True)

In [ ]:
from sklearn import preprocessing

In [ ]:
merged_df.columns

In [ ]:
merged_df['base_price'] = merged_df.price/merged_df.surge_multiplier

In [ ]:
names = ['distance', 'price', 'temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind', 'base_price', 'surge_multiplier']
for n in names:
    merged_df[f'n{n}'] = (merged_df[n] - merged_df[n].mean())/(merged_df[n].max() - merged_df[n].min())


In [ ]:
names = ['distance', 'price', 'temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind', 'base_price', 'surge_multiplier']
for n in names:
    merged_df[f'l{n}'] = np.log(merged_df[n])

In [ ]:
merged_df.describe()

In [ ]:
merged_df.drop(['lrain', 'lclouds'], axis=1, inplace=True)

In [ ]:
merged_df.info()

In [ ]:
merged_df.lsurge_multiplier.describe()

In [ ]:
merged_df.to_csv('data/merged_df_Tues.csv')

In [4]:
merged_df = pd.read_csv('data/merged_df_Tues.csv')

## Visualization

In [ ]:
merged_df.columns

In [ ]:
pd.plotting.scatter_matrix(merged_df[['lprice',
       'ldistance','lbase_price','ltemp']], alpha=.05)

In [ ]:
pd.plotting.scatter_matrix(merged_df[['lprice', 'lclouds',
       'lpressure', 'lrain', 'lhumidity', 'lwind']], alpha = .05)

In [ ]:
merged_df.corr()

In [ ]:
corr = merged_df[['distance', 'price', 'surge_multiplier', 'merge_date', 'temp',
       'location', 'clouds', 'pressure', 'rain', 'humidity', 'wind',
       'cab_type_Lyft', 'cab_type_Uber']].corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
corr = merged_df[['destination_Back Bay',
       'destination_Beacon Hill', 'destination_Boston University',
       'destination_Fenway', 'destination_Financial District',
       'destination_Haymarket Square', 'destination_North End',
       'destination_North Station', 'destination_Northeastern University',
       'destination_South Station', 'destination_Theatre District',
       'destination_West End']].corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

## OLS

### Spec 1: logged numerical and all raw one hot encoded dummies minus one

In [ ]:
merged_df.columns

In [ ]:
X = merged_df[['ldistance','ltemp', 'clouds',
       'lpressure', 'rain', 'lhumidity', 'lwind',
       'lsurge_multiplier', 'cab_type_Uber', 'destination_Back Bay',
       'destination_Boston University',
       'destination_Fenway', 'destination_Financial District',
       'destination_Haymarket Square', 'destination_North End',
       'destination_North Station', 'destination_Northeastern University',
       'destination_South Station', 'destination_Theatre District',
       'destination_West End', 'source_Back Bay', 
       'source_Boston University', 'source_Fenway',
       'source_Financial District', 'source_Haymarket Square',
       'source_North End', 'source_North Station',
       'source_Northeastern University', 'source_South Station',
       'source_Theatre District', 'source_West End', 'name_Black',
       'name_Black SUV', 'name_Lux', 'name_Lux Black', 'name_Lux Black XL',
       'name_Lyft', 'name_Lyft XL', 'name_Shared', 'name_Taxi',
       'name_UberPool', 'name_UberX', 'name_UberXL', 'day_Fri',
       'day_Mon', 'day_Sat', 'day_Thu', 'day_Tue', 'day_Wed',
       'AMPM_PM', 'month_Dec', 'week_47', 'week_48',
       'week_49', 'sports', 'wkndPM']]
X = sm.add_constant(X)
Y = merged_df['lprice']

In [10]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X,Y, test_size = .2, random_state=42)

In [ ]:
X_train1.shape

In [ ]:
X_test1.shape

In [ ]:
Y_train1.shape

In [ ]:
Y_test1.shape

In [ ]:
model = sm.OLS(Y_train1, X_train1).fit()

In [ ]:
model.summary()

In [ ]:
res = model.resid # residuals
fig = sm.qqplot(res, stats.t, fit=True, line=str(45))
plt.show()

## Lasso & Ridge

In [ ]:
from sklearn.linear_model import LassoCV
lasso1 = LassoCV()
lasso1.fit(X_train1, Y_train1)
preds1 = lasso1.predict(X_test)
lasso1.coef_

In [ ]:
lasso1.score(X_test1, Y_test1)

In [ ]:
lasso1_res = Y_test1 - preds1

In [ ]:
from sklearn.metrics import mean_squared_error
mse1 = mean_squared_error(Y_test1, preds1)

In [ ]:
np.exp(mse1)

In [ ]:
np.exp(Y_test1).mean()

In [ ]:
np.exp(mse1)/np.exp(Y_test1).mean()

In [ ]:
plt.hist(lasso1_res, bins=50)

In [ ]:
plt.scatter(Y_test1, preds1)

In [ ]:
import scipy.stats as stats
sm.qqplot(lasso1_res, stats.t, fit=True, line=str(45))

In [ ]:
plt.plot(Y_test1, lasso1_res, 'k.')

In [ ]:
X_train.loc[res[res > 0.15].index, :]

In [ ]:
from sklearn.linear_model import RidgeCV
ridge1 = RidgeCV()
ridge1.fit(X_train1, Y_train1)
preds1r = ridge1.predict(X_test1)
ridge1.coef_

In [ ]:
ridge1.score(X_test1, Y_test1)

In [ ]:
ridge1_res = Y_test1 - preds1r

In [ ]:
from sklearn.metrics import mean_squared_error
mse1r = mean_squared_error(Y_test1, preds1r)

In [ ]:
np.exp(mse1r)

In [ ]:
np.exp(Y_test1).mean()

In [ ]:
np.exp(mse1r)/np.exp(Y_test1).mean()

In [ ]:
plt.hist(ridge1_res, bins=50)

In [ ]:
plt.scatter(Y_test1, preds1r)

In [ ]:
sm.qqplot(ridge1_res, stats.t, fit=True, line=str(45))

### Spec 2: normalized numerical and all raw one hot encoded dummies minus one and rain dummy

In [ ]:
merged_df.columns

In [ ]:
X2 = merged_df[['ldistance','ltemp', 'clouds',
       'lpressure', 'rain', 'lhumidity', 'lwind',
       'lsurge_multiplier', 'cab_type_Uber', 'destination_Back Bay',
       'destination_Boston University',
       'destination_Fenway', 'destination_Financial District',
       'destination_Haymarket Square', 'destination_North End',
       'destination_North Station', 'destination_Northeastern University',
       'destination_South Station', 'destination_Theatre District',
       'destination_West End', 'source_Back Bay', 
       'source_Boston University', 'source_Fenway',
       'source_Financial District', 'source_Haymarket Square',
       'source_North End', 'source_North Station',
       'source_Northeastern University', 'source_South Station',
       'source_Theatre District', 'source_West End', 'name_Black',
       'name_Black SUV', 'name_Lux', 'name_Lux Black', 'name_Lux Black XL',
       'name_Lyft', 'name_Lyft XL', 'name_Shared', 'name_Taxi',
       'name_UberPool', 'name_UberX', 'name_UberXL', 'day_Fri',
       'day_Mon', 'day_Sat', 'day_Thu', 'day_Tue', 'day_Wed',
       'AMPM_PM', 'month_Dec', 'week_47', 'week_48',
       'week_49', 'sports', 'wkndPM', 'rain_d']]
X2 = sm.add_constant(X2)
Y2 = merged_df['lprice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2,Y2, test_size = .2, random_state=42)

In [ ]:
X_train2.shape

In [ ]:
X_test2.shape

In [ ]:
Y_train2.shape

In [ ]:
Y_test2.shape

In [ ]:
model2 = sm.OLS(Y_train2, X_train2).fit()

In [ ]:
model2.summary()

In [ ]:
from sklearn.linear_model import LassoCV
lasso2 = LassoCV()
lasso2.fit(X_train2, Y_train2)
preds2 = lasso2.predict(X_test2)
lasso2.coef_

In [ ]:
lasso2.score(X_test2, Y_test2)

In [ ]:
lasso2_res = Y_test2 - preds2

In [ ]:
from sklearn.metrics import mean_squared_error
mse2 = mean_squared_error(Y_test2, preds2)

In [ ]:
np.exp(mse2)

In [ ]:
np.exp(Y_test2).mean()

In [ ]:
np.exp(mse2)/np.exp(Y_test2).mean()

In [ ]:
plt.hist(Y_test2 - preds2, bins=50)

In [ ]:
plt.scatter(Y_test2, preds2)

In [ ]:
import scipy.stats as stats
sm.qqplot(lasso2_res, stats.t, fit=True, line=str(45))

In [ ]:
from sklearn.linear_model import RidgeCV
ridge2 = RidgeCV()
ridge2.fit(X_train2, Y_train2)
preds2r = ridge2.predict(X_test2)
ridge2.coef_

In [ ]:
ridge2.score(X_test2, Y_test2)

In [ ]:
ridge2_res = Y_test2 - preds2r

In [ ]:
from sklearn.metrics import mean_squared_error
mse2r = mean_squared_error(Y_test2, preds2r)

In [ ]:
np.exp(mse2r)

In [ ]:
np.exp(Y_test2).mean()

In [ ]:
np.exp(mse2r)/np.exp(Y_test2).mean()

In [ ]:
plt.hist(ridge2_res, bins=50)

In [ ]:
plt.scatter(Y_test2, preds2r)

In [ ]:
sm.qqplot(ridge2_res, stats.t, fit=True, line=str(45))

### Spec 3: normalized numerical and rain and selected dummies

In [ ]:
merged_df.columns

In [ ]:
X3 = merged_df[['ldistance','ltemp', 'clouds',
       'lpressure', 'rain', 'lhumidity', 'lwind',
       'lsurge_multiplier', 
       'cab_type_Uber', 'destination_Back Bay','source_Beacon Hill',
       'destination_Haymarket Square', 'destination_Northeastern University',
       'source_Back Bay', 'day_Fri',
       'AMPM_PM', 'month_Dec', 'sports', 'wkndPM', 'rain_d', 'Black']]
X3 = sm.add_constant(X3)
Y3 = merged_df['lprice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3,Y3, test_size = .2, random_state=42)

In [ ]:
X_train3.shape

In [ ]:
X_test3.shape

In [ ]:
Y_train3.shape

In [ ]:
Y_test3.shape

In [ ]:
model3 = sm.OLS(Y_train3, X_train3).fit()

In [ ]:
model3.summary()

In [ ]:
from sklearn.linear_model import LassoCV
lasso3 = LassoCV()
lasso3.fit(X_train3, Y_train3)
preds3 = lasso3.predict(X_test3)
lasso3.coef_

In [ ]:
lasso3.score(X_test3, Y_test3)

In [ ]:
lasso3_res = Y_test3 - preds3

In [ ]:
from sklearn.metrics import mean_squared_error
mse3 = mean_squared_error(Y_test3, preds3)

In [ ]:
np.exp(mse3)

In [ ]:
np.exp(Y_test3).mean()

In [ ]:
np.exp(mse3)/np.exp(Y_test3).mean()

In [ ]:
plt.hist(lasso3_res, bins=50)

In [ ]:
plt.scatter(Y_test3, preds3)

In [ ]:
import scipy.stats as stats
sm.qqplot(lasso3_res, stats.t, fit=True, line=str(45))

In [ ]:
from sklearn.linear_model import RidgeCV
ridge3 = RidgeCV()
ridge3.fit(X_train3, Y_train3)
preds3r = ridge3.predict(X_test3)
ridge3.coef_

In [ ]:
ridge3.score(X_test3, Y_test3)

In [ ]:
ridge3_res = Y_test3 - preds3r

In [ ]:
from sklearn.metrics import mean_squared_error
mse3r = mean_squared_error(Y_test3, preds3r)

In [ ]:
np.exp(mse3r)

In [ ]:
np.exp(Y_test3).mean()

In [ ]:
np.exp(mse3r)/np.exp(Y_test3).mean()

In [ ]:
plt.hist(ridge3_res, bins=50)

In [ ]:
plt.scatter(Y_test3, ridge3_res)

In [ ]:
sm.qqplot(ridge3_res, stats.t, fit=True, line=str(45))

### Spec 4: split black and not black

In [ ]:
X4 = merged_df[(merged_df.Black == 0)][['ldistance','ltemp', 'clouds',
       'lpressure', 'rain', 'lhumidity', 'lwind',
       'lsurge_multiplier', 
       'cab_type_Uber', 'destination_Back Bay','source_Beacon Hill',
       'destination_Haymarket Square', 'destination_Northeastern University',
       'source_Back Bay', 'day_Fri',
       'AMPM_PM', 'month_Dec', 'sports', 'wkndPM', 'rain_d']]
X4 = sm.add_constant(X4)
Y4 = merged_df[(merged_df.Black == 0)]['lprice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4,Y4, test_size = .2, random_state=42)

In [ ]:
X_train4.shape

In [ ]:
X_test4.shape

In [ ]:
Y_train4.shape

In [ ]:
Y_test4.shape

In [ ]:
model4 = sm.OLS(Y_train4, X_train4).fit()

In [ ]:
model4.summary()

In [ ]:
from sklearn.linear_model import LassoCV
lasso4 = LassoCV()
lasso4.fit(X_train4, Y_train4)
preds4 = lasso4.predict(X_test4)
lasso4.coef_

In [ ]:
lasso4.score(X_test4, Y_test4)

In [ ]:
lasso4_res = Y_test4 - preds4

In [ ]:
from sklearn.metrics import mean_squared_error
mse4 = mean_squared_error(Y_test4, preds4)

In [ ]:
np.exp(mse4)

In [ ]:
np.exp(Y_test4).mean()

In [ ]:
np.exp(mse4)/np.exp(Y_test4).mean()

In [ ]:
plt.hist(lasso4_res, bins=50)

In [ ]:
plt.scatter(Y_test4, preds4)

In [ ]:
import scipy.stats as stats
sm.qqplot(lasso4_res, stats.t, fit=True, line=str(45))

In [ ]:
plt.plot(Y_test4, lasso4_res, 'k.')

In [ ]:
from sklearn.linear_model import RidgeCV
ridge4 = RidgeCV()
ridge4.fit(X_train4, Y_train4)
preds4r = ridge4.predict(X_test4)
ridge4.coef_

In [ ]:
ridge4.score(X_test4, Y_test4)

In [ ]:
ridge_res4 = Y_test4 - preds4r

In [ ]:
from sklearn.metrics import mean_squared_error
mse4r = mean_squared_error(Y_test4, preds4r)

In [ ]:
np.exp(mse4r)

In [ ]:
np.exp(Y_test4).mean()

In [ ]:
np.exp(mse4r)/np.exp(Y_test4).mean()

In [ ]:
plt.hist(ridge_res4, bins=50)

In [ ]:
plt.scatter(Y_test4, preds4r)

In [ ]:
sm.qqplot(ridge_res4, stats.t, fit=True, line=str(45))

## Black

In [ ]:
X4b = merged_df[(merged_df.Black == 1)][['ldistance','ltemp', 'clouds',
       'lpressure', 'rain', 'lhumidity', 'lwind',
       'lsurge_multiplier', 
       'cab_type_Uber', 'destination_Back Bay','source_Beacon Hill',
       'destination_Haymarket Square', 'destination_Northeastern University',
       'source_Back Bay', 'day_Fri',
       'AMPM_PM', 'month_Dec', 'sports', 'wkndPM', 'rain_d']]
X4b = sm.add_constant(X4b)
Y4b = merged_df[(merged_df.Black == 1)]['lprice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train4b, X_test4b, Y_train4b, Y_test4b = train_test_split(X4b,Y4b, test_size = .2, random_state=42)

In [ ]:
X_train4b.shape

In [ ]:
X_test4b.shape

In [ ]:
Y_train4b.shape

In [ ]:
Y_test4b.shape

In [ ]:
model4b = sm.OLS(Y_train4b, X_train4b).fit()

In [ ]:
model4b.summary()

In [ ]:
from sklearn.linear_model import LassoCV
lasso4b = LassoCV()
lasso4b.fit(X_train4b, Y_train4b)
preds4b = lasso4b.predict(X_test4b)
lasso4b.coef_

In [ ]:
lasso4b.score(X_test4b, Y_test4b)

In [ ]:
lasso4b_res = Y_test4b - preds4b

In [ ]:
from sklearn.metrics import mean_squared_error
mse4b = mean_squared_error(Y_test4b, preds4b)

In [ ]:
np.exp(mse4b)

In [ ]:
np.exp(Y_test4b).mean()

In [ ]:
np.exp(mse4b)/np.exp(Y_test4b).mean()

In [ ]:
plt.hist(lasso4b_res, bins=50)

In [ ]:
plt.scatter(Y_test4b, preds4b)

In [ ]:
import scipy.stats as stats
sm.qqplot(lasso4b_res, stats.t, fit=True, line=str(45))

In [ ]:
plt.plot(Y_test4b, lasso4b_res, 'k.')

In [ ]:
from sklearn.linear_model import RidgeCV
ridge4b = RidgeCV()
ridge4b.fit(X_train4b, Y_train4b)
preds4br = ridge4.predict(X_test4b)
ridge4b.coef_

In [ ]:
ridge4b.score(X_test4b, Y_test4b)

In [ ]:
ridge_res4b = Y_test4b - preds4br

In [ ]:
from sklearn.metrics import mean_squared_error
mse4br = mean_squared_error(Y_test4b, preds4br)

In [ ]:
np.exp(mse4br)

In [ ]:
np.exp(Y_test4b).mean()

In [ ]:
np.exp(mse4br)/np.exp(Y_test4b).mean()

In [ ]:
plt.hist(ridge_res4b, bins=50)

In [ ]:
plt.scatter(Y_test4b, preds4br)

In [ ]:
sm.qqplot(ridge_res4b, stats.t, fit=True, line=str(45))

## Notes

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
embeded_lr_selector = SelectFromModel(LassoCV(), max_features=20)
embeded_lr_selector.fit(X_train, Y_train)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X_train.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

In [ ]:
embeded_lr_feature

In [ ]:
dataSet_df = merged_df[['distance', 'surge_multiplier', 'temp', 'clouds', 'pressure', 
                        'rain', 'humidity', 'wind', 'name_Black', 'name_Black SUV', 
                        'name_Lux', 'name_Lux Black', 'name_Lux Black XL', 'name_Lyft', 
                        'name_Lyft XL', 'name_Shared', 'name_Taxi', 'name_UberPool', 'name_UberX', 
                        'name_UberXL', 'name_WAV', 'destination_Back Bay', 'destination_Beacon Hill', 
                        'destination_Boston University', 'destination_Fenway', 'destination_Financial District',
                        'destination_Haymarket Square', 'destination_North End', 'destination_North Station', 
                        'destination_Northeastern University', 'destination_South Station', 
                        'destination_Theatre District', 'destination_West End', 'source_Back Bay', 
                        'source_Beacon Hill', 'source_Boston University', 'source_Fenway',
                        'source_Financial District', 'source_Haymarket Square',
                        'source_North End', 'source_North Station',
                        'source_Northeastern University', 'source_South Station',
                         'source_Theatre District', 'source_West End','price']]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
XX= dataSet_df[['distance', 'surge_multiplier', 'temp', 'clouds', 'pressure', 
                        'rain', 'humidity', 'wind', 'name_Black', 'name_Black SUV', 
                        'name_Lux', 'name_Lux Black', 'name_Lux Black XL', 'name_Lyft', 
                        'name_Lyft XL', 'name_Shared', 'name_Taxi', 'name_UberPool', 'name_UberX', 
                        'name_UberXL', 'name_WAV', 'destination_Back Bay', 'destination_Beacon Hill', 
                        'destination_Boston University', 'destination_Fenway', 'destination_Financial District',
                        'destination_Haymarket Square', 'destination_North End', 'destination_North Station', 
                        'destination_Northeastern University', 'destination_South Station', 
                        'destination_Theatre District', 'destination_West End', 'source_Back Bay', 
                        'source_Beacon Hill', 'source_Boston University', 'source_Fenway',
                        'source_Financial District', 'source_Haymarket Square',
                        'source_North End', 'source_North Station',
                        'source_Northeastern University', 'source_South Station',
                         'source_Theatre District', 'source_West End']]
YY = dataSet_df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(XX.values, YY.values, test_size=0.33, random_state=42)

In [ ]:
y_train.shape

In [ ]:
regr = RandomForestRegressor(max_depth=4, random_state=137, n_estimators=1000)
regr.fit(X_train, y_train)

In [ ]:
y_rf = regr.predict(X_test)

In [ ]:
fimpDict = {}
for fimp in list(zip(XX.columns, regr.feature_importances_)):
    print(fimp)
    fimpDict.update({fimp[0]:fimp[1]})

In [ ]:
kk = list(fimpDict.keys())
vv = list(fimpDict.values())


fimpDF = pd.DataFrame({'features':kk,
                      'importance':vv})









In [ ]:
fimpDF.sort_values(by='importance')

In [ ]:
plt.hist(y_test-y_rf, bins=50)

In [ ]:
regr2 = RandomForestRegressor(max_depth=5, random_state=137, n_estimators=2000)
regr2.fit(X_train, y_train)

In [ ]:
fimpDict2 = {}
for fimp in list(zip(XX.columns, regr2.feature_importances_)):
    print(fimp)
    fimpDict2.update({fimp[0]:fimp[1]})

In [ ]:
kk = list(fimpDict.keys())
vv = list(fimpDict.values())


fimpDF = pd.DataFrame({'features':kk,
                      'importance':vv})

In [ ]:
fimpDF.sort_values(by='importance')

## haven't run yet

In [ ]:
merged_df = pd.read_csv('merged_df.csv')

In [ ]:
merged_df = merged_df[(merged_df.price < 40) & (merged_df.price >5)]

In [ ]:
X = merged_df.drop(['price', 'location', 'merge_date',
                    'lprice', 'distance', 'cab_type_Lyft', 'destination_Back Bay',
                    'source_Back Bay', 'name_Taxi'], axis=1)
X = sm.add_constant(X)
X.reset_index(drop=True,inplace=True)
Y = pd.DataFrame(norm_Y.T)
Y.reset_index(drop=True,inplace=True)


In [ ]:
y_array = np.array(merged_df['price'])
norm_y = preprocessing.normalize([y_array])
merged_df['nnprice'] = pd.DataFrame(norm_y.T)

In [ ]:
d_array = np.array(merged_df['distance'])
norm_d = preprocessing.normalize([d_array])
merged_df['ndistance'] = pd.DataFrame(norm_d.T)

In [ ]:
t_array = np.array(merged_df['temp'])
norm_t = preprocessing.normalize([t_array])
merged_df['ntemp'] = pd.DataFrame(norm_t.T)

In [ ]:
c_array = np.array(merged_df['clouds'])
norm_c = preprocessing.normalize([c_array])
merged_df['nclouds'] = pd.DataFrame(norm_c.T)

In [ ]:
p_array = np.array(merged_df['pressure'])
norm_p = preprocessing.normalize([p_array])
merged_df['npressure'] = pd.DataFrame(norm_p.T)

In [ ]:
r_array = np.array(merged_df['rain'])
norm_r = preprocessing.normalize([r_array])
merged_df['nrain'] = pd.DataFrame(norm_r.T)

In [ ]:
h_array = np.array(merged_df['humidity'])
norm_h = preprocessing.normalize([h_array])
merged_df['nhumidity'] = pd.DataFrame(norm_h.T)

In [ ]:
w_array = np.array(merged_df['wind'])
norm_w = preprocessing.normalize([w_array])
merged_df['nwind'] = pd.DataFrame(norm_w.T)

## Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [49]:
lr_11 = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=3, max_iter=500, class_weight= 'balanced',)
lr_l2 = LogisticRegressionCV(penalty='l2', cv=3, max_iter=500 , class_weight= 'balanced')


In [49]:
lr_11_rs = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=3, max_iter=500, class_weight= 'balanced')
lr_l2_rs = LogisticRegressionCV(penalty='l2', cv=3, max_iter=500 , class_weight= 'balanced')


In [50]:
X = merged_df[['ldistance','ltemp', 'clouds',
       'lpressure', 'rain', 'lhumidity', 'lwind',
       'cab_type_Uber', 'destination_Back Bay',
       'destination_Boston University',
       'destination_Fenway', 'destination_Financial District',
       'destination_Haymarket Square', 'destination_North End',
       'destination_North Station', 'destination_Northeastern University',
       'destination_South Station', 'destination_Theatre District',
       'destination_West End', 'source_Back Bay', 
       'source_Boston University', 'source_Fenway',
       'source_Financial District', 'source_Haymarket Square',
       'source_North End', 'source_North Station',
       'source_Northeastern University', 'source_South Station',
       'source_Theatre District', 'source_West End', 'name_Black',
       'name_Black SUV', 'name_Lux', 'name_Lux Black', 'name_Lux Black XL',
       'name_Lyft', 'name_Lyft XL', 'name_Shared', 'name_Taxi',
       'name_UberPool', 'name_UberX', 'name_UberXL', 'day_Fri',
       'day_Mon', 'day_Sat', 'day_Thu', 'day_Tue', 'day_Wed',
       'AMPM_PM', 'month_Dec', 'week_47', 'week_48',
       'week_49', 'sports', 'wkndPM']]
X = sm.add_constant(X)
y = (merged_df.surge_multiplier != 1).astype(int)

/Users/ptw/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [ ]:
# ! pip install imblearn

In [51]:
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X, y, test_size=0.2, stratify = y, random_state=24)

In [52]:
X_train_lr.shape

(508193, 56)

In [53]:
y_train_lr.shape

(508193,)

In [54]:
y_train_lr.sum()

16707

In [33]:
16707/(508193 +16707)

0.03182891979424652

In [34]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
X_train_lr, y_train_lr = make_classification(
weights=[(1-0.03182891979424652),0.03182891979424652], n_samples=508193, random_state=10)
print('Original dataset shape %s' % Counter(y_train_lr))
#Original dataset shape Counter({1: 900, 0: 100})
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train_lr, y_train_lr)
print('Resampled dataset shape %s' % Counter(y_res))
#Resampled dataset shape Counter({0: 900, 1: 900})

Original dataset shape Counter({0: 492018, 1: 16175})
Resampled dataset shape Counter({0: 492018, 1: 492018})


In [ ]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_lr, y_train_lr) 
print(pd.Series(y_train_resampled).value_counts()) #Preview synthetic sample class distribution

In [43]:
y_res.sum()

492018

In [44]:
len(y_res)

984036

In [45]:
X_res

array([[ 0.44394941, -0.82096294,  0.08421542, ..., -1.83925523,
         0.0142563 , -1.15818895],
       [-1.1650269 ,  0.66919584,  1.28568035, ..., -0.91044961,
         0.7023219 ,  0.17114202],
       [ 0.53061168, -0.37411333, -1.10438309, ..., -0.25392073,
         0.26273281,  0.7347992 ],
       ...,
       [-0.76330262, -0.48496726, -0.43266563, ..., -0.90468592,
         0.71833107, -1.08092946],
       [ 1.00763883,  0.07307482, -0.82933737, ..., -1.5824914 ,
         2.22916499, -1.65733527],
       [-0.57769539, -0.87331701, -0.60782999, ...,  1.0582369 ,
         0.96258308,  0.87263583]])

In [ ]:
lr_11.fit(X_train_lr, y_train_lr)
lr_l2.fit(X_train_lr, y_train_lr)

/Users/ptw/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
lr_11_rs.fit(X_train_resampled, y_train_resampled)
lr_l2_rs.fit(X_train_resampled, y_train_resampled)

In [36]:
lr_11.score(X_test_lr, y_test_lr)

0.12005604136986517

In [37]:
lr_l2.score(X_test_lr, y_test_lr)

0.12005604136986517

In [38]:
lrl1_predict = lr_11.predict(X_test_lr)

In [39]:
lrl2_predict = lr_l2.predict(X_test_lr)

In [40]:
pred_comp_df = pd.DataFrame([lrl1_predict, lrl2_predict]).T

In [41]:
pred_comp_df[0].value_counts()

1    115973
0     11076
Name: 0, dtype: int64

In [47]:
pred_comp_df[1].value_counts()

1    115973
0     11076
Name: 1, dtype: int64

In [ ]:
lr.score(X_test_lr, y_test_lr)

In [46]:
lr_11.coef_

array([[ 1.26767738e-01,  9.33651929e-02,  7.19170866e-02,
         1.56491091e-01, -3.70477106e-02,  1.59259975e-01,
         3.65987952e-02, -7.25788113e-03, -1.01583444e-02,
        -2.41955590e-02,  1.50505581e-01, -9.21788006e-02,
         9.46629573e-03, -8.47026277e-02,  1.56904791e-01,
        -1.95840651e-01,  6.46528255e-02,  9.26780043e-02,
         1.84912457e-01,  6.44239631e-02,  8.89098930e-02,
        -1.99536458e-01,  1.23041745e-01, -6.47818080e-03,
         1.26760318e-01,  4.33513535e-01, -8.35720192e-02,
        -1.66845799e-02,  8.01683951e-02, -1.26965424e-01,
        -4.14342944e-02,  2.75065511e-02, -7.87246163e-02,
         3.33456999e-03,  2.65702870e-02, -1.58023600e-02,
         1.35063153e-01, -1.56109819e-01, -1.19888535e-01,
         3.71194184e-02,  1.12803089e-01,  1.21865396e+01,
        -7.56848746e+00, -6.27802907e-02,  7.01325220e-02,
        -1.61160602e-03, -8.58752259e-02,  3.13369215e-02,
         1.86573548e-01,  3.01091005e-02,  2.84874267e-0

In [48]:
lr_l2.coef_

array([[ 1.58949223e-01,  1.07029012e-01,  1.05240796e-01,
         1.81099296e-01, -5.55793263e-02,  1.79639071e-01,
         4.59531651e-02,  3.29606661e-03, -1.73515324e-02,
        -3.91446686e-02,  5.07516041e-01, -1.11596635e-01,
         7.10946644e-03, -1.00143197e-01,  1.92421179e-01,
        -2.41529924e-01,  7.61339925e-02,  9.69368342e-02,
         2.17961846e-01,  7.79267936e-02,  9.81895461e-02,
        -2.40272299e-01,  1.48019840e-01, -6.11987139e-04,
         1.51943901e-01, -2.50960586e+00, -8.80760614e-02,
        -2.26950992e-02,  9.64393282e-02, -1.77302712e-01,
        -5.30276797e-02,  2.46027021e-02, -1.08042824e-01,
         2.43856208e-02,  4.87024015e-02, -1.91734656e-02,
         1.68747320e-01, -1.90739918e-01, -1.40594472e-01,
         3.35954270e-02,  1.46172331e-01,  1.09332105e+01,
        -1.07411834e+01, -7.41909373e-02,  7.43379111e-02,
        -2.23918499e-03, -1.09993965e-01,  3.52700713e-02,
         2.16750882e-01,  3.77337960e-02,  3.62620127e-0

In [ ]:
lr_l2.coef_[abs(lr_l2.coef_)>.01]

In [ ]:
lr_11.coef_[abs(lr_11.coef_)>0]

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = lr.predict_proba(X_test_lr)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test_lr, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = lr_11.predict_proba(X_test_lr)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test_lr, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = lr_l2.predict_proba(X_test_lr)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test_lr, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
probs[:5]